# Spark Recommender System

In [4]:
# import necessary modules
import os
import shutil
import pyspark as ps
from pyspark.ml import Pipeline, Transformer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.sql import Row
from pyspark.sql.types import DoubleType

In [5]:
# create spark context
spark = (ps.sql.SparkSession.builder
        .appName("ALS model")
        .getOrCreate()
        )
sc = spark.sparkContext
print(spark.version)

3.5.0


## Read in model

We will use user clusters as part of user features for our model here. 

In [6]:
final_model = ALSModel.load("./model_5/bestmodel")


Py4JJavaError: An error occurred while calling o29.load.
: java.lang.UnsatisfiedLinkError: org.apache.hadoop.io.nativeio.NativeIO$POSIX.stat(Ljava/lang/String;)Lorg/apache/hadoop/io/nativeio/NativeIO$POSIX$Stat;
	at org.apache.hadoop.io.nativeio.NativeIO$POSIX.stat(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$POSIX.getStat(NativeIO.java:608)
	at org.apache.hadoop.fs.RawLocalFileSystem$DeprecatedRawLocalFileStatus.loadPermissionInfoByNativeIO(RawLocalFileSystem.java:934)
	at org.apache.hadoop.fs.RawLocalFileSystem$DeprecatedRawLocalFileStatus.loadPermissionInfo(RawLocalFileSystem.java:848)
	at org.apache.hadoop.fs.RawLocalFileSystem$DeprecatedRawLocalFileStatus.getPermission(RawLocalFileSystem.java:816)
	at org.apache.hadoop.fs.LocatedFileStatus.<init>(LocatedFileStatus.java:52)
	at org.apache.hadoop.fs.FileSystem$4.next(FileSystem.java:2199)
	at org.apache.hadoop.fs.FileSystem$4.next(FileSystem.java:2179)
	at org.apache.hadoop.mapred.FileInputFormat.singleThreaded

In [ ]:
# generate top_n product recommendations for user
nrecommend = 5
user_recs = final_model.recommendForAllUsers(nrecommend)
user_recs.show(4)

+--------------+--------------------+
|customer_index|     recommendations|
+--------------+--------------------+
|            26|[{12745, 0.997935...|
|            27|[{7645, 4.9884496...|
|            28|[{9397, 0.946731}...|
|            31|[{1693, 4.882163}...|
+--------------+--------------------+
only showing top 4 rows



In [ ]:
recs = user_recs.toPandas()

## Recommender Function

In [ ]:
import pandas as pd

user_features_df = pd.read_csv("./model_data/user_model.csv")
products = pd.read_csv("./model_data/item_model.csv")

In [ ]:
nrecommend = 5
def user_recommendations(user_id, top_n = 3):
    
    if top_n > nrecommend:
        print("Please select up to {} items to recommend".format(nrecommend))
        return; 
    
    prior_purchases = user_features_df[user_features_df['customer_unique_id'] == user_id]\
                                                                                        ['product_id'].unique()
    num_items = len(prior_purchases)
    
    if num_items < 3:
        items = num_items
    else:
        items = 3
    
    print("User: {}\n".format(user_id))
    print("Known positives: ")
    for n in range(items):
        known_like_product = user_features_df[user_features_df['customer_unique_id'] == user_id]\
                                                            ['product_id'].unique()[n]
        known_like_category = products[products['product_id'] == known_like_product]\
                                                            ['product_category_name'].unique()[0]
    
        print("\t", known_like_product)
        print("\t", known_like_category, "\n")
    
    
    customer_index = user_features_df[user_features_df['customer_unique_id'] == user_id]\
                                                            ['customer_index'].unique()[0]
    print("Top {} Recommendations: \n".format(top_n))
    rec_products = []
    
    for n in range(top_n):
        
        rec_products.append(list(recs[recs['customer_index'] == customer_index]['recommendations'])[0][n][0])
        
        print("{}.\n".format(n+1), products[products['product_index'] == rec_products[n]]\
                                                  [['product_id', 'product_category_name']].iloc[0][0])
        
        print(products[products['product_index'] == rec_products[n]]\
                                                  [['product_id', 'product_category_name']].iloc[0][1])

__Test for customer_id = 'c8ed31310fc440a3f8031b177f9842c3'__

In [ ]:
user_recommendations('c8ed31310fc440a3f8031b177f9842c3', top_n=5)

User: c8ed31310fc440a3f8031b177f9842c3

Known positives: 
	 1065e0ebef073787a7bf691924c60eeb
	 construction_tools_construction 

	 0cf2faf9749f53924cea652a09d8e327
	 construction_tools_construction 

	 0de59eddc63167215c972b0d785ffa7b
	 construction_tools_construction 

Top 5 Recommendations: 

1.
 3cc4d754aa53f733c4141170236195ec
computers_accessories
2.
 2c3dfe5b09e660349378f30d628d70bb
construction_tools_construction
3.
 f399bbedb1d21192dffefc4a8b30a229
housewares
4.
 d68eca92c5a12467b856479934dc39af
telephony
5.
 6b770b5934dbdb29064efdf34bfa6cf5
sports_leisure


__Test for customer_id = '698e1cf81d01a3d389d96145f7fa6df8'__

In [ ]:
user_recommendations('b56d31572e47b1e6d1b88d3128f2226b', top_n=5)

User: b56d31572e47b1e6d1b88d3128f2226b

Known positives: 
	 9d364fec2ac9a80f64bae58b5c034832
	 housewares 

Top 5 Recommendations: 

1.
 5dddb31154cbd968caa4706ef0f4e0f0
garden_tools
2.
 86df1e9a07c3d463f6a6c3f776051e9d
housewares
3.
 6a3370dbb5bd5817ac81a5db12cb2baa
perfumery
4.
 12a5914d1826c08632a428b769a4eb30
health_beauty
5.
 9c1e194db1d35a79d962ea610bfe0868
perfumery


__Test for customer_id = '89be58cbdd6ef318e3ed93fdb22be178'__

In [ ]:
user_recommendations('89be58cbdd6ef318e3ed93fdb22be178', top_n=5)

User: 89be58cbdd6ef318e3ed93fdb22be178

Known positives: 
	 3fdb534dccf5bc9ab0406944b913787d
	 diapers_and_hygiene 

Top 5 Recommendations: 

1.
 0723afa6f9a5a11c512396db0bb03051
stationery
2.
 0efaf4b155c6545bdc426ec0722cceaa
toys
3.
 743c14ed5f01c78944d28320cb2cb8e2
books_general_interest
4.
 551676a4458c024f741721013d87afc5
auto
5.
 fc8e852906ed3985a6e1ea67eb1dfdf9
furniture_decor


## Summary of Results

It's clear after producing a simple recommendation system with matrix factorization using only users prior purchase history that this dataset simply does not have the data necessary to give accurate results. Many of the attempted recommendations produced results that are clearly not relevant for the user. As can be seen in the suggestions above, many of the top items are from categories very different from the original purchase. 

Comparing these same recommendations with LightFM shows how well hybrid recommenders can do for data sets like this one, with very few return users. 